In [151]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings("ignore")

# 1. Load the Data

In [152]:
df = pd.read_csv(r"F:\mentorship program of upGrad\Online Retail2.csv", encoding = 'ISO-8859-1')

# 2. Understand the Data

## 2.1 top 5 rows

In [153]:
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


## 2.2 Last 5 rows

In [154]:
df.tail(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,09-12-2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,09-12-2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,09-12-2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,09-12-2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,09-12-2011 12:50,4.95,12680.0,France


## 2.3 Columns And Rows Count

In [155]:
rows, columns = df.shape
print(f'columns=', columns)
print(f'Rows=', rows)

columns= 8
Rows= 541909


## 2.4 Columns name

In [156]:
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ["Index", "Column Name"]

for index, col in enumerate(df.columns.to_list()):
    table.add_row([index, col])
print(table)

+-------+-------------+
| Index | Column Name |
+-------+-------------+
|   0   |  InvoiceNo  |
|   1   |  StockCode  |
|   2   | Description |
|   3   |   Quantity  |
|   4   | InvoiceDate |
|   5   |  UnitPrice  |
|   6   |  CustomerID |
|   7   |   Country   |
+-------+-------------+


## 2.5 INFO of DATASET

In [157]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## 2.6 Data Describe

In [158]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


## 2.7 Checking the missing DATA

In [159]:
# MISSING VALUE
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [160]:
# Missing Percentage
missing_percentage = (df.isnull().mean()*100).round(2)
missing_columns = missing_percentage[missing_percentage > 0]
print(missing_columns)

Description     0.27
CustomerID     24.93
dtype: float64


### Here <br> Description column having 0.27% values are missing <br> CustomerID column having 24.93% missing value.<br> Now we remove this missing value for quality analysis

In [161]:
# Before removing null/ misssing value create new DataFrame, Its helps avoid accidental data loss.
new_df = df.copy()

In [162]:
# Handling Missing Value
new_df = df.dropna(subset = ["Description", "CustomerID"])

In [163]:
# Now checking again null value
new_df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

## 2.8 Checking the Duplicate Value

In [164]:
# Duplicate value
new_df.duplicated().sum()

5225

In [165]:
# DataSet Contain 5225 values are duplicate we can remove it
new_df.drop_duplicates(inplace = True)
# checking again
new_df.duplicated().sum()

0

#### Now we have zero duplicate value

## 2.9 Now we are checking unique value for each column.

In [166]:
new_df[new_df["InvoiceNo"].str.contains("C", na=False)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,01-12-2010 09:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,01-12-2010 09:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,01-12-2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,01-12-2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,01-12-2010 10:24,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,09-12-2011 09:57,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,09-12-2011 10:28,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,09-12-2011 11:57,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,09-12-2011 11:58,1.25,17315.0,United Kingdom


In [167]:
new_df = new_df[~new_df['InvoiceNo'].astype(str).str.contains('C')]
new_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [168]:
new_df.shape

(392732, 8)

### "The dataset has been refined by removing canceled orders, ensuring that only valid transactions are included for further analysis."

In [169]:
# checking agai shape of dataset
new_df.shape

(392732, 8)

### "The dataset has been successfully cleaned, ensuring all missing values, duplicate value & cancelled orders are addressed, and it is now ready for further analysis."

# 3. TRANSFORM THE DATA

In [170]:
# Creating new feature
new_df["TotalPrice"] = new_df["Quantity"]*new_df["UnitPrice"]

In [171]:
new_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,09-12-2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,09-12-2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,09-12-2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,09-12-2011 12:50,4.15,12680.0,France,16.60


In [172]:
# Create a new feature Date
new_df["InvoiceDate"] = pd.to_datetime(new_df["InvoiceDate"], errors='coerce', dayfirst=True)
new_df['Date'] = new_df['InvoiceDate'].dt.date

In [173]:
# Standardize numeric features
scaler = StandardScaler()
numeric_cols = ["Quantity", "UnitPrice"]
new_df[numeric_cols] = scaler.fit_transform(new_df[numeric_cols])

In [174]:
# checking shape of data again
new_df.shape

(392732, 10)

In [186]:
#checking outlier
#Q1 = new_df['TotalPrice'].quantile(0.25)
#Q3 = new_df['TotalPrice'].quantile(0.75)
#IQR = Q3 - Q1

#lower_bond = Q1 - 1.5*IQR
#upper_bond = Q1 + 1.5*IQR
#outlier = new_df[(new_df['TotalPrice'] < lower_bond) | (new_df['TotalPrice'] > upper_bond)]
#print(outlier)

# 4. Recency, Frequency & monetery Analysis

## 4.1 Recency

In [175]:
# Group the dataset by customerID and find the most recent purchase date for each customer
recency_df = new_df.groupby('CustomerID')['Date'].max().reset_index().rename(columns = {'Date': 'LastPurchaseDate'})

# Display the first few rows of the dataframe to inspect the results
recency_df.head()

,CustomerID,LastPurchaseDate
0,12346.0,2011-01-18
1,12347.0,2011-12-07
2,12348.0,2011-09-25
3,12349.0,2011-11-21
4,12350.0,2011-02-02


In [176]:
# latest date of purchasing in dataset
latest_date = recency_df['LastPurchaseDate'].max()
print(latest_date)

2011-12-09


In [177]:
# Calculate the recency of each customer's last purchase
recency_df['Recency'] = recency_df['LastPurchaseDate'].apply(lambda x : (latest_date - x).days)
# Display the first 10 rows of the recency dataframe
recency_df.head(10)

,CustomerID,LastPurchaseDate,Recency
0,12346.0,2011-01-18,325
1,12347.0,2011-12-07,2
2,12348.0,2011-09-25,75
3,12349.0,2011-11-21,18
4,12350.0,2011-02-02,310
5,12352.0,2011-11-03,36
6,12353.0,2011-05-19,204
7,12354.0,2011-04-21,232
8,12355.0,2011-05-09,214
9,12356.0,2011-11-17,22


In [178]:
# Drop the 'LastPurchaseDate' column from the dataframe
recency_df.drop('LastPurchaseDate', axis = 1, inplace = True)
recency_df.head()

,CustomerID,Recency
0,12346.0,325
1,12347.0,2
2,12348.0,75
3,12349.0,18
4,12350.0,310


## 4.2 Frequency

In [179]:
# Group the data by customer ID and count the number of invoices for each customer
frequency_df = df.groupby('CustomerID')['InvoiceNo'].count().reset_index().rename(columns = {'InvoiceNo': 'Frequency'})

# Display the first 10 rows of the dataframe to inspect the results
frequency_df.head(10)

,CustomerID,Frequency
0,12346.0,2
1,12347.0,182
2,12348.0,31
3,12349.0,73
4,12350.0,17
5,12352.0,95
6,12353.0,4
7,12354.0,58
8,12355.0,13
9,12356.0,59


## 4.3 Monetary

In [180]:
# Group the data by customer ID and sum the total amount spent by each customer
monetary_df = new_df.groupby('CustomerID')['TotalPrice'].sum().reset_index().rename(columns = {'TotalPrice': 'MonetaryValue'})
monetary_df.head()

,CustomerID,MonetaryValue
0,12346.0,77183.60
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40


### 4.4 Merging RFM DATA

In [182]:
RFM_df = recency_df.merge(frequency_df, on="CustomerID", how = "inner").merge(monetary_df, on="CustomerID", how = "inner")
RFM_df.head()

,CustomerID,Recency,Frequency,MonetaryValue
0,12346.0,325,2,77183.60
1,12347.0,2,182,4310.00
2,12348.0,75,31,1797.24
3,12349.0,18,73,1757.55
4,12350.0,310,17,334.40


In [183]:
RFM_df.shape

(4339, 4)